In [1]:
import pandas as pd
import json

In [2]:
df1 = pd.read_csv("../Data/cowin_vaccine_data_districtwise.csv", skiprows=[1])
df1 = df1.groupby(['State_Code', 'State', 'District_Key', 'District'], as_index=False).agg('sum')
jData = open('neighbor-districts.json', 'r').read()
obj = json.loads(jData)

In [3]:
data = {'district_id': [item for item in obj]}            #Dictionary for final output
for i in range(12):                                       #To make all list of the dictionary of same size
    data[str(i)] = [None for item in obj]
for idx, item in enumerate(obj):                          #To add all the neighbours of the districts
    l = 0
    for itm in obj[item]:
        data[str(l)][idx] = itm
        l += 1
df2 = pd.DataFrame(data)

In [4]:
def getName(df, idx):                                    #To get the name of the district in the index idx
    return df['District_Key'][idx]

def getIndex(df, name):                                  #To get the index of the district "name" in the dataframe
    name = name[:name.find('/')]
    return df.index[df['District'].str.lower() == name.lower()].tolist()

In [5]:
for item in obj:
    idx = getIndex(df1, item)
    if len(idx) == 1:
        df2.replace(item, getName(df1, idx[0]), inplace = True)
    else:
        sc = df1['State_Code'][getIndex(df1, obj[item][0])[0]]
        if sc == df1["State_Code"][idx[0]]:
            df2.replace(item, getName(df1, idx[0]), inplace = True)
        else:
            df2.replace(item, getName(df1, idx[1]), inplace = True)
df2.sort_values('district_id', inplace = True)

In [6]:
newJson = {}
for idx, rows in df2.iterrows():
    temp_list = []
    i = 0
    while rows[str(i)] != None:
        temp_list.append(rows[str(i)])
        i += 1
    newJson[rows['district_id']] = temp_list

In [7]:
with open('../Final Outputs/neighbor-districts-modified.json', 'w') as f:
    json.dump(newJson, f, indent = 4)